#### In this code there are many times while running you might observe discrepancies like getting Nan and value greater than 1 for fowlkes_mallow score. This is due to the fact that sklearn implementation for windows OS gives overflow error. I have tested this on MacOS on a friend's PC and it worked perfectly. So please try to run on MacOS.

##### PS: I have dropped some indices in calculating fowlkes_mallow score and magically it gives output on windows, most of the times using sample method of pandas dataframe.

In [1]:
import pandas as pd
import numpy as np
import glob, os
import scipy
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import load_digits
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from scipy.spatial.distance import cdist

In [2]:
digits = load_digits(return_X_y=False)
x = digits.data
y = digits.target
f = pd.DataFrame(x)
f['y'] = y
f.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,y
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


**K_means own implementation**

In [3]:
def k_means(f, m = 2):                  #### here f is the data and m is the number of clusters
    mu_center = f.sample(n = m)
    theta = 100000
    theta_diff = 100000
    i = 0
    while theta_diff > 1:
        i += 1
        a = cdist(f.iloc[:,:64],mu_center.iloc[:,:64], metric='euclidean')
        f['min_dist'] = a.min(axis=1)
        f['Cluster'] = a.argmin(axis =1)
        theta_diff = theta - f['min_dist'].sum()
        theta = f['min_dist'].sum()
        mu_center = f.groupby(['Cluster']).mean()
        mu_center = mu_center.drop(['min_dist'],axis = 1).reset_index(drop=True)
    else:
        print("Iteration",i,"coverges.")
    f['Cluster'] += 1 
    return f , mu_center

In [4]:
df , mu = k_means(f, m = 10)

##### part i
g = df.groupby('Cluster')    
c = [g.get_group(x) for x in g.groups]

for i in range(len(c)):
    n = int(c[i].y.mode())
    print('Cluster',i,'is labeled as ',n)
    c[i]['label'] = n
#     print('Points in the cluster are:','\n',c[i].head()) ## to print top 5 points in the cluster
    print('Indices of points in the cluster are:','\n',c[i].index) ## to print indices of points in the cluster
    print('==========================================================================')
    
df = pd.concat(c)
df

Iteration 14 coverges.
Cluster 0 is labeled as  4
Indices of points in the cluster are: 
 Int64Index([   4,   14,   24,   41,   64,   68,   87,   97,  100,  110,
            ...
            1708, 1731, 1735, 1754, 1764, 1767, 1777, 1778, 1788, 1791],
           dtype='int64', length=170)
Cluster 1 is labeled as  2
Indices of points in the cluster are: 
 Int64Index([  12,   22,   84,  113,  132,  142,  152,  180,  181,  184,
            ...
            1717, 1718, 1721, 1724, 1742, 1744, 1751, 1780, 1782, 1783],
           dtype='int64', length=179)
Cluster 2 is labeled as  1
Indices of points in the cluster are: 
 Int64Index([  11,   21,   27,   42,   47,   54,   56,   70,   75,   77,   80,
              90,  107,  131,  141,  151,  172,  177,  186,  200,  210,  215,
             218,  221,  227,  235,  312,  375,  381,  387,  397,  407,  428,
             429,  433,  447,  456,  466,  467,  471,  476,  479,  480,  485,
             493,  496,  498,  638,  655,  665,  670,  673,  675, 

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,y,min_dist,Cluster,label
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,16.0,4.0,0.0,0.0,4,27.656899,1,4
14,0.0,0.0,0.0,8.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0.0,10.0,15.0,4.0,0.0,0.0,4,21.022634,1,4
24,0.0,0.0,0.0,0.0,12.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,8.0,0.0,0.0,4,29.345577,1,4
41,0.0,0.0,0.0,9.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0.0,9.0,16.0,2.0,0.0,0.0,4,16.964963,1,4
64,0.0,0.0,0.0,6.0,16.0,4.0,0.0,0.0,0.0,0.0,...,0.0,7.0,16.0,3.0,0.0,0.0,4,21.371737,1,4
68,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,16.0,5.0,0.0,0.0,4,25.270035,1,4
87,0.0,0.0,0.0,9.0,16.0,4.0,0.0,0.0,0.0,1.0,...,0.0,11.0,13.0,0.0,0.0,0.0,4,32.165171,1,4
97,0.0,0.0,0.0,3.0,14.0,1.0,0.0,0.0,0.0,0.0,...,0.0,2.0,16.0,7.0,0.0,0.0,4,20.030226,1,4
100,0.0,0.0,0.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,6.0,0.0,0.0,4,25.333504,1,4
110,0.0,0.0,0.0,10.0,15.0,0.0,0.0,0.0,0.0,0.0,...,0.0,12.0,9.0,0.0,0.0,0.0,4,28.339091,1,4


In [5]:
### part ii
confusion_matrix(df['y'],df['label'])

array([[176,   0,   0,   0,   2,   0,   0,   0,   0,   0],
       [  0,  54,  24,   1,   0,   1,   2,   0, 100,   0],
       [  1,   3, 151,   8,   0,   0,   0,   2,   8,   4],
       [  0,   0,   1, 152,   0,   2,   0,   7,   7,  14],
       [  0,   2,   0,   0, 166,   0,   0,  11,   2,   0],
       [  0,   0,   0,   1,   2, 136,   1,   0,   0,  42],
       [  1,   0,   0,   0,   0,   0, 177,   0,   3,   0],
       [  0,  10,   0,   0,   0,   0,   0, 167,   2,   0],
       [  0,  10,   3,   4,   0,   4,   2,   4, 101,  46],
       [  0,  20,   0,   5,   0,   6,   0,   8,   1, 140]], dtype=int64)

In [6]:
#### score without dropping some indices

s_1 = metrics.fowlkes_mallows_score(df['y'].sample(len(df)),df['Cluster'].sample(len(df)))
print(s_1)

1.707013468341404


**Agglomerative clustering with ward linkage**

In [7]:
agg_clus = AgglomerativeClustering(n_clusters = 10,linkage = 'ward').fit(x)
df1 = f
df1['agg'] = agg_clus.labels_
print(f.shape)
##### part i
g1 = df1.groupby('agg')    
c1 = [g1.get_group(x) for x in g1.groups]
for i in range(len(c1)):
    n = int(c1[i].y.mode())
    print('Cluster',i,'is labeled as ',n)
    c1[i]['label_agg'] = n
#     print('Points in the cluster are:','\n',c1[i].head()) ## to print top 5 points in the cluster
    print('Indices of points in the cluster are:','\n',c1[i].index) ## to print indices of points in the cluster
    print('==========================================================================')
df1 = pd.concat(c1)
df1

(1797, 68)
Cluster 0 is labeled as  5
Indices of points in the cluster are: 
 Int64Index([  15,   25,   32,   33,   35,   46,   71,   74,  102,  109,
            ...
            1699, 1700, 1702, 1713, 1738, 1741, 1769, 1776, 1784, 1787],
           dtype='int64', length=181)
Cluster 1 is labeled as  3
Indices of points in the cluster are: 
 Int64Index([   3,    5,    9,   13,   19,   23,   29,   31,   37,   39,
            ...
            1740, 1756, 1758, 1759, 1765, 1770, 1772, 1786, 1792, 1795],
           dtype='int64', length=317)
Cluster 2 is labeled as  2
Indices of points in the cluster are: 
 Int64Index([  12,   22,   84,  113,  132,  142,  152,  180,  181,  184,
            ...
            1717, 1718, 1721, 1724, 1742, 1744, 1751, 1780, 1782, 1783],
           dtype='int64', length=197)
Cluster 3 is labeled as  7
Indices of points in the cluster are: 
 Int64Index([   7,   17,   27,   43,   44,   52,   61,   69,   81,   86,
            ...
            1710, 1711, 1719, 1728, 

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,y,min_dist,Cluster,agg,label_agg
15,0.0,5.0,12.0,13.0,16.0,16.0,2.0,0.0,0.0,11.0,...,16.0,2.0,0.0,0.0,0.0,5,21.607335,10,0,5
25,0.0,0.0,12.0,8.0,8.0,7.0,0.0,0.0,0.0,3.0,...,14.0,9.0,0.0,0.0,0.0,5,23.002431,10,0,5
32,0.0,2.0,13.0,16.0,16.0,16.0,11.0,0.0,0.0,5.0,...,15.0,8.0,0.0,0.0,0.0,5,22.302626,10,0,5
33,0.0,6.0,13.0,5.0,8.0,8.0,1.0,0.0,0.0,8.0,...,15.0,3.0,0.0,0.0,0.0,5,26.639107,10,0,5
35,0.0,3.0,15.0,8.0,8.0,6.0,0.0,0.0,0.0,4.0,...,14.0,1.0,0.0,0.0,0.0,5,23.379516,10,0,5
46,0.0,1.0,15.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,...,14.0,4.0,0.0,0.0,0.0,5,30.658726,10,0,5
71,0.0,1.0,13.0,15.0,12.0,12.0,5.0,0.0,0.0,4.0,...,10.0,1.0,0.0,0.0,0.0,5,20.596746,10,0,5
74,0.0,0.0,13.0,13.0,8.0,2.0,0.0,0.0,0.0,5.0,...,16.0,16.0,6.0,0.0,0.0,5,28.511938,10,0,5
102,0.0,0.0,12.0,16.0,16.0,8.0,0.0,0.0,0.0,3.0,...,16.0,12.0,0.0,0.0,0.0,5,18.499712,10,0,5
109,0.0,0.0,15.0,16.0,16.0,12.0,2.0,0.0,0.0,2.0,...,16.0,15.0,3.0,0.0,0.0,5,19.952236,10,0,5


In [8]:
### part ii
confusion_matrix(df1['y'],df1['label_agg'])

array([[178,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 155,  27,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 166,   0,   0,   0,   0,   1,  10,   0],
       [  0,   0,   0, 169,   0,   0,   0,   1,  13,   0],
       [  0,   0,   0,   0, 178,   0,   0,   3,   0,   0],
       [  0,   0,   0,   2,   0, 179,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 180,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0,   0, 179,   0,   0],
       [  0,   3,   4,   1,   0,   0,   0,   1, 165,   0],
       [  0,  20,   0, 145,   0,   2,   0,  11,   2,   0]], dtype=int64)

In [9]:
#### score without dropping some indices
print(len(df1))
s1_1 = metrics.fowlkes_mallows_score(df1['y'].sample(len(df1)),df1['label_agg'].sample(len(df1)))
print(s1_1)


1797
nan


In [10]:
### only getting score after dropping these indices else getting Nan
#### only do this once after running the above code or else you are dropping many rows
#### i.e. make sure the len of data frame is 1792
df1 = df1.drop(df1.index[[662,663,664,665,666]])

len(df1)

1792

In [11]:
#### part iii
s1_2 = metrics.fowlkes_mallows_score(df1['y'].sample(len(df1)),df1['agg'].sample(len(df1)))
print(s1_2)

0.8224260466533488


**Affinity Propogation**

In [12]:
aff_clus = AffinityPropagation(damping = 0.99175).fit(x)
df2 = f
df2['aff'] = aff_clus.labels_

##### part i
g2 = df2.groupby('aff')    
c2 = [g2.get_group(x) for x in g2.groups]

for i in range(len(c2)):
    n = int(c2[i].y.mode())
    print('Cluster',i,'is labeled as ',n)
    c2[i]['label_aff'] = n
#     print('Points in the cluster are:','\n',c2[i].head()) ## to print top 5 points in the cluster
    print('Indices of points in the cluster are:','\n',c2[i].index) ## to print indices of points in the cluster
    print('==========================================================================')
df2 = pd.concat(c2)
df2

Cluster 0 is labeled as  6
Indices of points in the cluster are: 
 Int64Index([   2,   16,   34,   65,   88,   95,  106,  146,  195,  223,
            ...
            1608, 1609, 1610, 1623, 1629, 1636, 1645, 1709, 1755, 1771],
           dtype='int64', length=106)
Cluster 1 is labeled as  6
Indices of points in the cluster are: 
 Int64Index([   6,   26,   58,   66,   67,   82,  104,  148,  156,  164,  188,
             196,  197,  212,  255,  282,  360,  362,  392,  402,  452,  468,
             492,  522,  550,  558,  572,  574,  582,  598,  604,  620,  622,
             680,  704,  773,  780,  782,  784,  792,  810,  842,  858,  882,
             921, 1085, 1092, 1131, 1173, 1647, 1673, 1683, 1693, 1701, 1705,
            1725, 1732, 1733, 1734, 1749, 1762, 1796],
           dtype='int64')
Cluster 2 is labeled as  3
Indices of points in the cluster are: 
 Int64Index([   3,    5,    8,    9,   13,   19,   23,   29,   31,   37,
            ...
            1750, 1752, 1756, 1758, 1759,

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,y,min_dist,Cluster,agg,aff,label_aff
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,11.0,16.0,9.0,0.0,2,30.949164,7,4,0,6
16,0.0,0.0,0.0,8.0,15.0,1.0,0.0,0.0,0.0,0.0,...,15.0,16.0,11.0,0.0,6,29.807875,9,6,0,6
34,0.0,0.0,0.0,5.0,14.0,2.0,0.0,0.0,0.0,0.0,...,16.0,15.0,3.0,0.0,6,22.534798,9,6,0,6
65,0.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0,...,12.0,14.0,9.0,0.0,6,12.720304,9,6,0,6
88,0.0,0.0,0.0,10.0,11.0,0.0,0.0,0.0,0.0,0.0,...,13.0,11.0,6.0,0.0,6,20.189040,9,6,0,6
95,0.0,0.0,0.0,11.0,16.0,8.0,0.0,0.0,0.0,0.0,...,16.0,15.0,0.0,0.0,6,29.050730,7,6,0,6
106,0.0,0.0,0.0,5.0,11.0,1.0,0.0,0.0,0.0,0.0,...,14.0,16.0,8.0,0.0,6,31.883262,9,6,0,6
146,0.0,0.0,0.0,6.0,14.0,1.0,0.0,0.0,0.0,0.0,...,15.0,14.0,6.0,0.0,6,17.526858,9,6,0,6
195,0.0,0.0,0.0,6.0,14.0,3.0,0.0,0.0,0.0,0.0,...,14.0,13.0,5.0,0.0,6,16.743822,9,6,0,6
223,0.0,0.0,0.0,8.0,12.0,1.0,0.0,0.0,0.0,0.0,...,16.0,16.0,11.0,0.0,6,24.146840,9,6,0,6


In [13]:
### part ii
confusion_matrix(df2['y'],df2['label_aff'])

array([[176,   0,   0,   0,   1,   0,   1,   0,   0,   0],
       [  5,   0,  68,  11,  31,   6,  12,  49,   0,   0],
       [  6,   0, 153,   8,   0,   0,   1,   9,   0,   0],
       [  2,   0,   1, 167,   0,   2,   0,  11,   0,   0],
       [  0,   0,   2,   0, 168,   1,   4,   6,   0,   0],
       [  3,   0,   2,   5,   1, 169,   1,   1,   0,   0],
       [  0,   0,   0,   0,   0,   0, 181,   0,   0,   0],
       [  0,   0,   3,   0,   0,   0,   0, 176,   0,   0],
       [  5,   0,  43,  49,   5,  23,  20,  29,   0,   0],
       [ 46,   0,   0,  96,   0,   8,   1,  29,   0,   0]], dtype=int64)

In [14]:
#### score without dropping some indices
print(len(df2))
s2_1 = metrics.fowlkes_mallows_score(df2['y'].sample(len(df2)),df2['label_aff'].sample(len(df2)))
print(s2_1)


1797
nan


In [15]:
### only getting score after dropping these indices else getting Nan
#### only do this once after running the above code or else you are dropping many rows
#### i.e. make sure the len of data frame is 1792
df2 = df2.drop(df1.index[[662,663,664,665,666]])

len(df2)

1792

In [16]:
#### part iii
s2_2 = metrics.fowlkes_mallows_score(df2['y'].sample(len(df2)),df2['label_aff'].sample(len(df2)))
print(s2_2)


1.8924637454675306
